# Benchmark algorithm for isomorphism of OA's

#### Scheme

As in "Classifier":
 - Generate one OA to test against
 -  generate a dataset of OA's, balanced 50% iso and 50% non-iso
 - measure the performance of the algorithm
 - Test against the computational complexity of other (exact) approaches
    

### Remarks:
As per the theory, it only makes sense to use **switch** isomorphisms, so at most there are $2^d$ isomorphs of each array. It makes no sense to use "sampleChildren" larger than this

Imports

In [1]:
import os
import os.path

import numpy as np
from statistics.pmf import PMF
from statistics.OA import OA, genOA, genFiltrations
from statistics.OAIso import allIsomorphisms, allSwitchIsomorphisms
from persistence.pers import compute_perm_classes, persistencePairs, WasDistances, BotDistances, Was1DDistances, DistancesDistr
from persistence.distances import wasserstein2DNoDiag, wasserstein1D

#These are not necessary
#from geometry.polytope import Polytope, computePolytope
#from geometry.polytope import conditionsOA3, conditionsOA4, conditionsOA5

import random

#from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt
import pandas as pd

### Parameters

In [2]:
N = 24
d = 5


sampleChildren = 500
sampleChildren = 32
loop = 500


### Complexity Comparison

We are now sure that the normal form of on OA gives a unique representative of its iso class. It is computed by the LMC test (lexicographically minimum by columns), described in Schoen, Eendebak, Nguyen (2009). Its complexity is not straightforward, but we could estimate that for the positive case (array is minimal) it is of the same order as $d! 2^2$. It is implemented in the `oapackage` library as `reduceLMCform( arr )`.

Our test has a tunable complexity, so in case the exact complexity is impractical, we could still provide as answer with a certain amount of confidence. We can showcase it by describing the accuracy curve as a function of complexity (% of correct guesses as a function of how many loop iterations). Notice we do at most $2^d$ iterations (number of switch isomorphisms) (in fact we could remove one as we do not use the identity). 

One goal would be to identify whether the exact complexity is greater or smaller than $2^d$. The other one would be to idenify if, for example, the 50-50 threshold (which is equivalent to chance for a binary test) sits at a sufficiently small fraction of the complexity of the exact test. 